In [2]:
import pandas as pd
import numpy as np
import time_series_transform.base as base
from time_series_transform.time_series_transformer import *

In [3]:
df = pd.read_csv('./Data/Train/sales_train_validation.csv')
df.index = df.id
df = df.drop('id',axis=1)

In [4]:
CATEGORICAL_DIM = [
    'item_id','dept_id','cat_id','store_id','state_id'
]
WINDOW_SIZE = 20
TEST_RANGE = WINDOW_SIZE + 20

In [5]:
testDate = list(map(lambda x: 'd_'+str(x),list(range(1913-TEST_RANGE,1914))))
trainDateDrop = list(map(lambda x: 'd_'+str(x),list(range(1,1600))))
test = df[CATEGORICAL_DIM+testDate]
train = df.drop(testDate+trainDateDrop,axis = 1)
SEQ_TRANSFORMER = []

In [6]:
T_SEQUENCE_SIZE = len(train.drop(CATEGORICAL_DIM,axis =1).columns)-WINDOW_SIZE +1

In [13]:
ptt = Pandas_Time_Series_Dataset(train)
for i in CATEGORICAL_DIM:
    ptt.set_config(name = i,
                   colNames = [i],
                   tensorType = 'category',
                   param = {"windowSize":0,"batchSize":T_SEQUENCE_SIZE,"outType":np.float32},
                   sequence_stack = None,
                   isResponseVar = False
                  )

In [14]:
gen = ptt.make_data_generator()

In [16]:
pass

In [12]:
ptt.config

{'item_id': {'colNames': 'item_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'dept_id': {'colNames': 'dept_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'cat_id': {'colNames': 'cat_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'store_id': {'colNames': 'store_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False},
 'state_id': {'colNames': 'state_id',
  'tensorType': 'category',
  'param': {'windowSize': 0, 'batchSize': 254, 'outType': numpy.float32},
  'sequence_stack': None,
  'responseVariable': False}}

In [ ]:
sells_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,2),name = 'sells')
# state_input = tf.keras.layers.Input(shape=1,name = 'state_id')
# store_input = tf.keras.layers.Input(shape=1,name = 'store_id')
# item_input =  tf.keras.layers.Input(shape=1,name = 'item_id')
# cat_input = tf.keras.layers.Input(shape=1,name = 'cat_id')
# dept_input = tf.keras.layers.Input(shape=1,name = 'dept_id')
# price_input = tf.keras.layers.Input(shape=1,name = 'price')

# weekDay_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,1),name = 'weekDay')
# month_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,1),name= 'month')
# snapca_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,1),name = 'snapca')
# snapwi_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,1),name = 'snapwi')
# snaptx_input = tf.keras.layers.Input(shape=(WINDOW_SIZE,1),name = 'snaptx')


# state  = tf.keras.layers.Embedding(embeddingNum['state_id'],WINDOW_SIZE,name = 'state')(state_input)
# store  = tf.keras.layers.Embedding(embeddingNum['store_id'],WINDOW_SIZE,name ='store')(store_input)
# item  = tf.keras.layers.Embedding(embeddingNum['item_id'],WINDOW_SIZE,name = 'item')(item_input)
# dept  = tf.keras.layers.Embedding(embeddingNum['dept_id'],WINDOW_SIZE, name = 'dept')(dept_input)
# cat  = tf.keras.layers.Embedding(embeddingNum['cat_id'],WINDOW_SIZE,name = 'cat')(cat_input)

# price = tf.keras.layers.Embedding(5,WINDOW_SIZE,name = 'price_emb')(price_input)

# month = tf.keras.layers.Flatten()(month_input)
# month = tf.keras.layers.Embedding(13,1,
#                                   input_length= WINDOW_SIZE,
#                                   name = 'month_emb')(month)

# weekDay = tf.keras.layers.Flatten()(weekDay_input)
# weekDay = tf.keras.layers.Embedding(8,8
#                                     ,input_length= WINDOW_SIZE,
#                                     name ='weekDay_emb')(weekDay)

# snapca = tf.keras.layers.Flatten()(snapca_input)
# snapca = tf.keras.layers.Embedding(2,2,
#                                    input_length= WINDOW_SIZE,
#                                    name = 'snapca_emb')(snapca)


# snaptx = tf.keras.layers.Flatten()(snaptx_input)
# snaptx = tf.keras.layers.Embedding(2,2,
#                                    input_length= WINDOW_SIZE,
#                                    name ='snaptx_emb')(snaptx)


# snapwi = tf.keras.layers.Flatten()(snapwi_input)
# snapwi = tf.keras.layers.Embedding(2,2,
#                                    input_length= WINDOW_SIZE,
#                                    name ='snapwi_emb')(snapwi)


# price = tf.keras.layers.Reshape((WINDOW_SIZE,1))(price)
# state  = tf.keras.layers.Reshape((WINDOW_SIZE,1))(state)
# store  = tf.keras.layers.Reshape((WINDOW_SIZE,1))(store)
# item  = tf.keras.layers.Reshape((WINDOW_SIZE,1))(item)
# dept  = tf.keras.layers.Reshape((WINDOW_SIZE,1))(dept)
# cat  = tf.keras.layers.Reshape((WINDOW_SIZE,1))(cat)




# lstm = tf.keras.layers.Concatenate()([
#     snapwi,
#     snaptx,
#     snapca,
#     sells_input,
#     month,
#     weekDay,
#     price,
#     state,
#     store,
#     item,
#     dept,
#     cat
# ])

dense = tf.keras.layers.LSTM(1)(sells_input)

# price = tf.keras.layers.Flatten()(price)
# state  = tf.keras.layers.Flatten()(state)
# store  = tf.keras.layers.Flatten()(store)
# item  = tf.keras.layers.Flatten()(item)
# dept  = tf.keras.layers.Flatten()(dept)
# cat  = tf.keras.layers.Flatten()(cat)
# loc = tf.keras.layers.Concatenate(name = 'location')([state,store])
# product = tf.keras.layers.Concatenate(name = 'product')([item,dept,cat,price])
# cate = tf.keras.layers.Concatenate()([product,loc,lstm])
# cate = tf.keras.layers.Dense(10,'relu')(lstm)

# dense  = tf.keras.layers.Dense(1)(lstm)


lstm_embed = tf.keras.models.Model({
    'sells':sells_input,
#     'state_id':state_input,
#     'store_id':store_input,
#     'item_id':item_input,
#     'dept_id':dept_input,
#     'cat_id':cat_input,
#     'weekDay':weekDay_input,
#     'price':price_input,
#     'snapca':snapca_input,
#     'snapwi':snapwi_input,
#     'snaptx':snaptx_input,
#     'month':month_input
},dense)


rmsp = tf.keras.optimizers.RMSprop(lr = 0.2,
                                   decay = 1e-5,
                               )

rmse = tf.keras.metrics.RootMeanSquaredError()
lstm_embed.compile(optimizer=rmsp, 
                   loss='mse',
                   metrics = ['mae','mse',rmse
                             ]
                  )